📘 CatBoost 이론 요약

---

**CatBoost (Categorical Boosting)**는 

    - 범주형 특성을 자동으로 처리하는 기능이 내장된 부스팅 알고리즘입니다. 

    - 다른 모델과 달리 전처리 과정이 간소화되어 편리합니다.

---

주요 특징:

**범주형 특성 자동 처리**: One-Hot Encoding과 같은 수동 전처리 없이도 `모델이 자체적으로 범주형 특성을 효율적으로 학습`합니다.

**순서형 타겟 인코딩**: 범주형 변수를 타겟 변수와의 관계를 기반으로 인코딩하여 `정보 손실을 최소화`합니다.

**과적합 방지**: 순서형 부스팅을 사용하여 `과적합을 효과적으로 방지`합니다.

---


In [1]:
# CatBoost 모델 실습 코드

# 1. 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

print("CatBoost 모델 실습 시작...")

# 2. 데이터셋 로드
# scikit-learn에서 제공하는 붓꽃(Iris) 데이터셋 사용
# 붓꽃 데이터에는 범주형 특성이 없지만, CatBoost 사용법을 익히기 위해 진행합니다.
iris = load_iris()
X, y = iris.data, iris.target

print(f"데이터셋 로드 완료. 특성(X)의 shape: {X.shape}, 레이블(y)의 shape: {y.shape}")

# 3. 학습 및 테스트 데이터 분리
# 전체 데이터의 80%를 학습용, 20%를 테스트용으로 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

# 4. CatBoost 모델 정의 및 파이프라인 생성
# CatBoost Classifier 모델 정의
# silent=True: 학습 과정의 로그를 출력하지 않음
# random_state: 재현성을 위한 시드값
catboost_model = CatBoostClassifier(silent=True, random_state=42)

# 전처리와 모델 학습을 한 번에 처리하는 파이프라인 구축
# StandardScaler: 특성을 표준화하여 모델 학습 효율을 높임
# classifier: CatBoost 모델
pipeline = Pipeline(steps=[('scaler', StandardScaler()),
                           ('classifier', catboost_model)])

print("\n--- CatBoost 모델 학습 시작 ---")

# 5. 파이프라인을 사용하여 모델 학습
pipeline.fit(X_train, y_train)

print("--- 학습 완료 ---")

# 6. 테스트 데이터로 예측 및 정확도 평가
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nCatBoost 테스트 정확도: {accuracy:.4f}")

# 7. 교차 검증을 통한 모델 성능 평가
# cv=5: 5개의 폴드로 나누어 교차 검증 진행
cv_scores = cross_val_score(pipeline, X, y, cv=5)

print("\n--- 교차 검증 결과 ---")
print(f"각 폴드별 정확도: {cv_scores}")
print(f"교차 검증 평균 정확도: {cv_scores.mean():.4f} (표준편차: {cv_scores.std():.4f})")

print("\nCatBoost 모델 실습 완료.")


CatBoost 모델 실습 시작...
데이터셋 로드 완료. 특성(X)의 shape: (150, 4), 레이블(y)의 shape: (150,)
학습 데이터 크기: (120, 4)
테스트 데이터 크기: (30, 4)

--- CatBoost 모델 학습 시작 ---
--- 학습 완료 ---

CatBoost 테스트 정확도: 0.9333

--- 교차 검증 결과 ---
각 폴드별 정확도: [0.96666667 0.96666667 0.93333333 0.96666667 1.        ]
교차 검증 평균 정확도: 0.9667 (표준편차: 0.0211)

CatBoost 모델 실습 완료.


---

### 📊 CatBoost 모델 실습 결과 분석

---

1.  **테스트 정확도**
    * **CatBoost 테스트 정확도: `0.9333`**
    * 이는 `30개`의 테스트 데이터 중 약 `28개`(0.9333 * 30 ≈ 28)를 정확하게 예측했다는 의미입니다. 
    * XGBoost와 동일한 테스트 정확도를 기록했습니다.

2.  **교차 검증 결과**
    * **각 폴드별 정확도**: `[0.9666, 0.9666, 0.9333, 0.9666, 1.0000]`
    * **평균 정확도**: `0.9667`
    * **표준편차**: `0.0211`
    * 교차 검증 평균 정확도가 `0.9667`로, XGBoost(`0.9533`)나 LightGBM(`0.9467`)보다 높게 나왔습니다. 
    * 이는 CatBoost가 붓꽃 데이터셋에서 `가장 뛰어난 일반화 성능`을 보였다는 것을 의미합니다.

---

### 부스팅 모델 3가지 최종 비교

이제 세 가지 부스팅 모델의 실습 결과를 종합적으로 비교해 볼까요?

| 구분 | **XGBoost** | **LightGBM** | **CatBoost** |
| :--- | :--- | :--- | :--- |
| **테스트 정확도** | 0.9333 | `0.9667` | 0.9333 |
| **교차 검증 평균 정확도** | 0.9533 | 0.9467 | `0.9667` |
| **교차 검증 표준편차** | `0.0163` | 0.0452 | 0.0211 |

* **성능**: **CatBoost**가 교차 검증 평균 정확도에서 가장 높은 점수를 얻으며 `가장 뛰어난 성능`을 보였습니다.
* **안정성**: **XGBoost**가 가장 낮은 표준편차를 기록하여 `가장 안정적인 모델`임을 입증했습니다.
* **속도**: LightGBM의 경우, 로그에는 표시되지 않았지만 리프 중심 성장 방식을 통해 `다른 모델보다 빠르게 학습을 완료했을 가능성`이 높습니다.

붓꽃 데이터셋의 경우 모든 모델이 꽤 높은 성능을 보였지만, 결과적으로 `CatBoost가 가장 뛰어난 일반화 성능`을 보여준 것을 확인할 수 있습니다.